In [1]:
import pandas as pd
import pyranges as pr
import sys
import os
from Bio import SearchIO
import yaml
from snakemake.io import expand
import pdb
import swan_vis as swan

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *
from scripts.mane_utils import *

In [2]:
config_file = '../snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [3]:
ab = '../'+expand(config['data']['ab'], species='human')[0]
filt_ab = '../'+expand(config['data']['filt_ab'], species='human')[0]
read_annot = '../'+expand(config['data']['read_annot'], species='human')[0]
ref_t_metadata = '../'+expand(config['ref']['t_info'], species='human')[0]
ref_g_metadata = '../'+expand(config['ref']['g_info'], species='human')[0]
t_metadata = '../'+expand(config['ref']['cerberus']['t_info'], species='human')[0]
lib_meta = '../'+expand(config['data']['meta'], species='human')[0]
swan_file = '../'+expand(config['data']['sg'], species='human')[0]
cerberus_h5 = '../'+expand(config['data']['cerb_annot'], species='human')[0]
cerb_t_metadata = '../'+expand(config['data']['t_info'], species='human')[0]
major_isos = '../'+expand(config['data']['major_isos'], species='human', obs_col='sample')[0]
pp_summary = '../'+expand(config['data']['p_pred']['summary'], species='human')[0]

ver = 'v40_cerberus'
min_tpm = 1
gene_subset = 'polya'
obs_col = 'sample'

m_lib_meta = '../'+expand(config['data']['meta'], species='mouse')[0]
m_ab = '../'+expand(config['data']['ab'], species='mouse')[0]
m_filt_ab = '../'+expand(config['data']['filt_ab'], species='mouse')[0]
m_read_annot = '../'+expand(config['data']['read_annot'], species='mouse')[0]
m_ref_t_metadata = '../'+expand(config['ref']['t_info'], species='mouse')[0]
m_ref_g_metadata = '../'+expand(config['ref']['g_info'], species='mouse')[0]
m_t_metadata = '../'+expand(config['ref']['cerberus']['t_info'], species='mouse')[0]
m_lib_meta = '../'+expand(config['data']['meta'], species='mouse')[0]
m_swan_file = '../'+expand(config['data']['sg'], species='mouse')[0]
m_cerberus_h5 = '../'+expand(config['data']['cerb_annot'], species='mouse')[0]
m_cerb_t_metadata = '../'+expand(config['data']['t_info'], species='mouse')[0]
m_major_isos = '../'+expand(config['data']['major_isos'], species='mouse', obs_col='sample')[0]
m_pp_summary = '../'+expand(config['data']['p_pred']['summary'], species='mouse')[0]


mouse_ver = 'vM25_cerberus'

## Save TPMs of transcripts expressed in 16 cell lines

In [108]:
datasets = get_ljungman_datasets()

ab_df = pd.read_csv(filt_ab, sep='\t')
ab_df, _ = get_tpm_table(ab_df,
               how='iso',
               min_tpm=min_tpm,
               gene_subset=gene_subset)
ab_df = ab_df[datasets]
ab_df.to_csv('transcript_abundance.tsv', sep='\t')

Calculating iso TPM values
Subsetting for polya genes
Enforcing minimum TPM
Total # isos detected: 231136
# isos >= 1 tpm: 210239
Applying gene type and novelty subset
Number of isos reported: 206806


,huvec_1_1,huvec_1_2,hmec_1_1,hmec_1_2,panc1_1_1,pc3_1_1,k562_1_1,hepg2_1_1,hct116_1_1,imr90_1_1,...,ocily7_1_1,ocily7_1_2,mcf10a_1_1,mcf10a_1_2,pc9_1_1,pc9_1_2,a673_1_1,a673_1_2,calu3_1_1,calu3_1_2
annot_transcript_id,,,,,,,,,,,,,,,,,,,,,
"ENSG00000227232[1,2,1]",0.57069,1.138878,0.000000,0.000000,0.0,0.875113,0.000000,1.226145,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.834033,0.0,0.856421
"ENSG00000227232[1,3,1]",0.57069,0.569439,0.000000,0.584375,0.0,0.875113,0.815517,0.000000,0.000000,3.919458,...,1.064405,0.0,0.0,0.0,0.821249,2.578556,0.0,0.000000,0.0,1.712842
"ENSG00000227232[1,4,1]",0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.815517,0.000000,3.183814,1.306486,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000
"ENSG00000227232[1,5,1]",0.00000,0.569439,1.086655,0.000000,0.0,0.000000,0.000000,0.000000,1.061271,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.856421
"ENSG00000227232[2,3,1]",0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.856421


## Get transcripts expressed in 16 cell lines

In [38]:
# get tids that are expressed in ljungman datasets
datasets = get_ljungman_datasets()
df = pd.read_csv(filt_ab, sep='\t')
df = get_det_table(df,
               how='iso',
               min_tpm=min_tpm,
               gene_subset=gene_subset)

Calculating iso TPM values
Subsetting for polya genes


/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2096: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[tpm_col] = (df[d]*1000000)/df[total_col]
/Users/fairliereese/Documents/programming/mortazavi_lab/data/paper_rnawg/scripts/utils.py:2095: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[total_col] = df[d].sum()


Enforcing minimum TPM
Total # isos detected: 231136
# isos >= 1 tpm: 210239
Applying gene type and novelty subset
Number of isos reported: 206806
Found 138 total libraries


In [39]:
# limit to 16 cell lines
df = df.loc[datasets]

In [40]:
df = df.transpose()
df = df.loc[df.sum(axis=1)>=1]

In [41]:
tids = df.index.tolist()

## Get v29 transcripts

In [42]:
ca = cerberus.read(cerberus_h5)

In [43]:
ref_df = ca.t_map.loc[ca.t_map.source=='v29']
ref_tids = ref_df.transcript_id.unique().tolist()

## Find transcripts w/ evidence of fusion

In [4]:
sg = swan.read(swan_file)

Read in graph from ../data/human/swan_graph.p


In [91]:
# get the edges for each transcript
# get the genes associated with each edge
# get the transcripts that have edges assc. w/ more than 1 gene

In [49]:
# get a list of exons / introns
df = swan.pivot_path_list(sg.t_df, 'path')
df.reset_index(inplace=True)
df = df.merge(sg.t_df[['gid', 'gname']].reset_index(), on='tid', how='left')

In [50]:
# get a table of edge:gid
eg_df = df[['edge_id', 'gid']].drop_duplicates()

In [51]:
# merge in the edge:gid to the tid:edge table
df.drop(['gid', 'gname'], axis=1, inplace=True)
df = df.merge(eg_df[['edge_id', 'gid']], how='left', on='edge_id')

In [52]:
# how many genes are represented by the edges of each transcript?
gt_df = df.drop('edge_id', axis=1).copy(deep=True)
gt_df = gt_df.groupby('tid').nunique().rename({'gid':'n_genes'}, axis=1).reset_index()

# get gene ids and gene names assc. with this as well and merge in
temp = df.drop('edge_id', axis=1).copy(deep=True)
temp = temp.merge(sg.t_df[['gname', 'gid']].drop_duplicates(), how='left', on='gid').drop_duplicates()
temp = temp.groupby('tid').agg(','.join).reset_index()
gt_df = gt_df.merge(temp, on='tid', how='left')

# limit to transcripts w/ >1 gene
gt_df = gt_df.loc[gt_df.n_genes>1]

In [53]:
# limit entries in to those expressed in the 16 cell lines
gt_df = gt_df.loc[gt_df.tid.isin(tids)]

# label v29 annotated transcripts
gt_df['Known'] = gt_df.tid.isin(ref_tids)

In [54]:
gt_df.to_csv('fusion_transcripts.tsv', sep='\t', index=False)

## Get all unspliced transcripts w/ gene name, gene id

In [44]:
# label monoexonic transcripts
t_df = sg.t_df.copy(deep=True)
t_df = t_df[['tid', 'gid', 'gname', 'path']].reset_index(drop=True)
t_df['unspliced'] = t_df.apply(lambda x: len(x.path)==1, axis=1)
t_df = t_df.loc[t_df.unspliced==True]
t_df.set_index('tid', inplace=True)

# get edges used in each monoexonic transcript
ue_t_df = swan.pivot_path_list(t_df, path_col='path')
ue_df = sg.edge_df.loc[ue_t_df.edge_id.tolist()]

# add coords for these edges
ue_df = ue_df.merge(sg.loc_df[['chrom', 'coord']],
            how='left', left_on='v1', right_on='vertex_id')
ue_df.rename({'coord': 'start'}, axis=1, inplace=True)
ue_df = ue_df.merge(sg.loc_df[['coord']],
            how='left', left_on='v2', right_on='vertex_id')
ue_df.rename({'coord': 'stop'}, axis=1, inplace=True)
ue_df.drop(['v1', 'v2'], axis=1, inplace=True)

# merge info in with transcript info
ue_t_df.reset_index(inplace=True)
ue_t_df = ue_t_df.merge(ue_df, on='edge_id', how='left')
ue_t_df = ue_t_df.merge(sg.t_df.reset_index(drop=True)[['tid', 'gid', 'gname']], on='tid', how='left')

In [46]:
# limit entries in to those expressed in the 16 cell lines
ue_t_df = ue_t_df.loc[ue_t_df.tid.isin(tids)]

# label v29 annotated transcripts
ue_t_df['Known'] = ue_t_df.tid.isin(ref_tids)

In [47]:
ue_t_df.to_csv('unspliced_transcripts.tsv', sep='\t', index=False)

,tid,edge_id,strand,edge_type,annotation,chrom,start,stop,gid,gname,Known
0,"ENSG00000001460[12,13,1]",947924,-,exon,False,chr1,24357356,24356949,ENSG00000001460,STPG1,False
3,"ENSG00000002834[9,9,1]",974058,+,exon,False,chr17,38919325,38921820,ENSG00000002834,LASP1,False
4,"ENSG00000003249[5,6,1]",339329,-,exon,True,chr16,90007811,90004821,ENSG00000003249,DBNDD1,True
5,"ENSG00000004399[17,17,1]",997925,-,exon,False,chr3,129555524,129555164,ENSG00000004399,PLXND1,False
6,"ENSG00000004487[11,18,1]",287,+,exon,True,chr1,23080823,23083739,ENSG00000004487,KDM1A,True


## 230516 Find transcripts with evidence of gene fusion

In [ ]:
sg = swan.read(swan_file)

In [ ]:
# get a list of splice junctions
df = swan.pivot_path_list(sg.t_df, 'path')
df.reset_index(inplace=True)
df = df.merge(sg.t_df[['gid', 'gname']].reset_index(), on='tid', how='left')

# limit to those from transcripts in 16 cell lines
print(len(df.index))
df = df.loc[df.tid.isin(tids)]
print(len(df.index))

# remove transcripts already annotated
df = df.loc[~df.tid.isin(ref_tids)]
print(len(df.index))

In [ ]:
# edges that occur in more than one gene
df2 = df[['edge_id', 'gid']].groupby(['edge_id']).nunique().reset_index().rename({'gid':'n_genes'}, axis=1)

In [ ]:
df3 = df2.loc[df2.n_genes>1]

In [ ]:
sns.displot(df3, x='n_genes', kind='hist', discrete=True)

In [ ]:
# get names of transcripts / genes that have more than 1 
# gene that share the same splice junction
print(len(df3.index))
df3 = df3.merge(df, on='edge_id', how='left')
print(len(df3.index))

In [ ]:
df3.head()

In [ ]:
# add coords for these edges
temp = sg.edge_df.merge(sg.loc_df[['chrom', 'coord']],
            how='left', left_on='v1', right_on='vertex_id')
temp.rename({'coord': 'start'}, axis=1, inplace=True)
temp = temp.merge(sg.loc_df[['coord']],
            how='left', left_on='v2', right_on='vertex_id')
temp.rename({'coord': 'stop'}, axis=1, inplace=True)
temp.drop(['v1', 'v2'], axis=1, inplace=True)
df3 = df3.merge(temp, how='left', on='edge_id')

In [78]:
gnames = df3.gname.unique().tolist()
df3.loc[df3.gname==gnames[8]].head()

,edge_id,n_genes,tid,gid,gname,strand,edge_type,annotation,chrom,start,stop
843,17886,2,"ENSG00000143369[1,1,2]",ENSG00000143369,ECM1,+,intron,True,chr1,150508279,150509531
845,17887,2,"ENSG00000143369[1,1,2]",ENSG00000143369,ECM1,+,exon,True,chr1,150509531,150509581
847,17888,2,"ENSG00000143369[1,1,2]",ENSG00000143369,ECM1,+,intron,True,chr1,150509581,150509661
849,17889,2,"ENSG00000143369[1,1,2]",ENSG00000143369,ECM1,+,exon,True,chr1,150509661,150509762
851,17890,2,"ENSG00000143369[1,1,2]",ENSG00000143369,ECM1,+,intron,True,chr1,150509762,150509922


In [82]:
df3.head()

,edge_id,n_genes,tid,gid,gname,strand,edge_type,annotation,chrom,start,stop
0,2130,2,"ENSG00000058673[1,17,1]",ENSG00000058673,ZC3H11A,+,intron,True,chr1,203803016,203816926
1,2130,2,"ENSG00000058673[1,18,1]",ENSG00000058673,ZC3H11A,+,intron,True,chr1,203803016,203816926
2,2130,2,"ENSG00000058673[1,19,1]",ENSG00000058673,ZC3H11A,+,intron,True,chr1,203803016,203816926
3,2130,2,"ENSG00000058673[1,2,1]",ENSG00000058673,ZC3H11A,+,intron,True,chr1,203803016,203816926
4,2130,2,"ENSG00000058673[1,20,1]",ENSG00000058673,ZC3H11A,+,intron,True,chr1,203803016,203816926


In [85]:
datasets = get_ljungman_datasets()

ab_df = pd.read_csv(filt_ab, sep='\t')
ab_df, _ = get_tpm_table(ab_df,
               how='iso',
               min_tpm=min_tpm,
               gene_subset=gene_subset)
print(ab_df.shape)
ab_df = ab_df.loc[tids, datasets]
print(ab_df.shape)

Calculating iso TPM values
Subsetting for polya genes
Enforcing minimum TPM
Total # isos detected: 231136
# isos >= 1 tpm: 210239
Applying gene type and novelty subset
Number of isos reported: 206806


(206806, 138)
(2989, 24)


In [99]:
ab_df.to_csv('novel_fusions_transcript_tpm.tsv', sep='\t')

In [6]:
ab_df = pd.read_csv('novel_fusions_transcript_tpm.tsv', sep='\t')
ab_df.head()

,annot_transcript_id,huvec_1_1,huvec_1_2,hmec_1_1,hmec_1_2,panc1_1_1,pc3_1_1,k562_1_1,hepg2_1_1,hct116_1_1,...,ocily7_1_1,ocily7_1_2,mcf10a_1_1,mcf10a_1_2,pc9_1_1,pc9_1_2,a673_1_1,a673_1_2,calu3_1_1,calu3_1_2
0,"ENSG00000058673[1,17,1]",0.00000,0.000000,0.000000,0.584375,0.00000,0.0,0.815517,0.0,2.122543,...,4.257620,3.604625,0.000000,1.409037,0.821249,0.000000,1.064829,3.336130,0.000000,0.856421
1,"ENSG00000058673[1,18,1]",1.71207,2.277755,1.629982,2.337500,0.00000,0.0,1.631034,0.0,1.061271,...,11.708455,8.410793,4.300363,4.931630,0.821249,4.297593,6.388973,5.004195,3.677955,8.564210
2,"ENSG00000058673[1,19,1]",0.57069,0.569439,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,...,0.000000,1.201542,1.433454,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"ENSG00000058673[1,2,1]",0.00000,0.000000,0.543327,0.000000,1.01183,0.0,0.000000,0.0,1.061271,...,2.128810,2.403084,3.583636,0.704519,4.106243,2.578556,1.064829,5.004195,0.919489,0.856421
4,"ENSG00000058673[1,20,1]",0.00000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,...,4.257620,1.201542,1.433454,3.522593,0.821249,0.859519,0.000000,0.834033,0.000000,0.000000


In [100]:
# table for Ariel - edge that's shared along with tids, gids, and gnames
df3 = df3[['edge_id', 'edge_type', 'chrom', 'strand', 'start', 'stop', 'tid', 'gid', 'gname']]
df3 = df3.drop_duplicates()

# one for tids
tid_df = df3.drop(['gname', 'gid'], axis=1)
tid_df = tid_df.groupby(['edge_id', 'edge_type', 'chrom', 'strand', 'start', 'stop']).agg(','.join).reset_index()

# one for gids
gid_df = df3.drop('tid', axis=1)
gid_df.drop_duplicates(inplace=True)
gid_df = gid_df.groupby(['edge_id', 'edge_type', 'chrom', 'strand', 'start', 'stop']).agg(','.join).reset_index()

# merge in 
tid_df = tid_df.merge(gid_df, how='left', on=['edge_id', 'edge_type', 'chrom', 'strand', 'start', 'stop'])

tid_df.to_csv('novel_fusion_edges.tsv', sep='\t')

In [101]:
tid_df.head()

,edge_id,edge_type,chrom,strand,start,stop,tid,gid,gname
0,2130,intron,chr1,+,203803016,203816926,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
1,2131,exon,chr1,+,203816926,203817124,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
2,2132,intron,chr1,+,203817124,203818570,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
3,2133,exon,chr1,+,203818570,203818689,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
4,2134,intron,chr1,+,203818689,203828299,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"


## 230517

In [4]:
edge_df = pd.read_csv('novel_fusion_edges.tsv', sep='\t')
edge_df.head()

,Unnamed: 0,edge_id,edge_type,chrom,strand,start,stop,tid,gid,gname
0,0,2130,intron,chr1,+,203803016,203816926,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
1,1,2131,exon,chr1,+,203816926,203817124,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
2,2,2132,intron,chr1,+,203817124,203818570,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
3,3,2133,exon,chr1,+,203818570,203818689,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
4,4,2134,intron,chr1,+,203818689,203828299,"ENSG00000058673[1,17,1],ENSG00000058673[1,18,1...","ENSG00000058673,ENSG00000257315","ZC3H11A,ZBED6"
